In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

In [3]:
train=pd.read_csv('/content/drive/MyDrive/colab전용/Auction_master_train.csv')
test=pd.read_csv('/content/drive/MyDrive/colab전용/Auction_master_test.csv')
submission=pd.read_csv('/content/drive/MyDrive/colab전용/Auction_submission.csv')

데이터 전처리

Auction_key 경매 아파트 고유 키값  
Auction_class 경매구분 강제경매 : 법원에 소송을 제기하여 판결을 얻은 후 집행권원에 따라 경매를 진행 임의경매 : 등기부등본 담보권(저당권, 가압류등)으로 경매를 진행    
Bid_class 입찰구분(일반/개별/일괄)  
Claim_price 경매 신청인의 청구 금액  
Appraisal_company 감정사  
Appraisal_date 감정일자    
Auction_count 총경매횟수   
Auction_miscarriage_count 총유찰횟수   
Total_land_gross_area 총토지전체면적(㎡)  
Total_land_real_area 총토지실면적(㎡)   
Total_land_auction_area 총토지경매면적(㎡)   
Total_building_area 총건물면적(㎡)   
Total_building_auction_area 총건물경매면적(㎡)  
Total_appraisal_price 총감정가   
Minimum_sales_price 최저매각가격, 입찰 시 입찰인이 최저로 제시해야만 하는 금액   
First_auction_date 최초경매일    
Final_auction_date 최종경매일   
Final_result 최종결과    
Creditor 채권자, 경매 신청인   
addr_do 주소_시도    
addr_si 주소_시군구    
addr_dong 주소_읍면동   
addr_li 주소_리     
addr_san 주소_산번지 여부(Y)    
addr_bunji1 주소_번지1   
addr_bunji2 주소_번지2    
addr_etc 주소_기타주소    
Apartment_usage 건물(토지)의 대표 용도   
Completion_date 준공일    
Preserve_regist_date 보존등기일, 건물을 신축하고 처음으로 등기   
Total_floor 총층수   
Current_floor 현재층수    
Specific 기타_특이사항    
Share_auction_YorN 지분경매 여부(Y), 하나의 부동산이 전체가 아닌 일부만 경매가 진행 (하나의 부동산의 공유자가 지분으로 소유권을 가지고 있을 때 그중 일부 지분만 경매가 진행)     
road_name 도로명주소_도로명    
road_bunji1 도로명주소_번지1   
road_bunji2 도로명주소_번지2    
Close_date 종국일자     
Close_result 종국결과, 낙찰과 배당의 차이 : 경매 진행은 ①경매진행(낙찰) ▷ ②낙찰허가결정 ▷ ③대금납부 ▷ ④배당 후 종결 로 이뤄집니다. 낙찰자가 최고가로 입찰(①)해서 낙찰허가(②)를 받으면 대금납부기한 낙찰금을 입금(③)합니다. 법원은 납부된 낙찰금을 가지고 채권자에게 순위에 의한 배당(④)을 해주고 경매는 종결됩니다.     
point.y 위도    
point.x 경도   
Hammer_price 낙찰가   

방법 1. 숫자 열로만 해보기  
방법 2. 숫자 열 중 상관성이 많이 없는걸로만 해보기   
방법 3. 임의,강제 같은 경우에 따라 바뀔 수 있으니 범주형 포함해서 해보기  
방법 4. 월별 Hammer_price 보고, 시간도 변수 넣어서 해보기   
방법 5. 아파트 정보(돈, 위치, 공부해서 더 보기.)  

In [4]:
# 방법 1.
tmp_train=train.select_dtypes(exclude='object')
tmp_test=test.select_dtypes(exclude='object')

In [5]:
# nan 값. 처리
cols=['addr_bunji1','addr_bunji2','road_bunji1','road_bunji2']

tmp_train=tmp_train.drop(cols,axis=1)
tmp_test=tmp_test.drop(cols,axis=1)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [7]:
# 11개의 특성과 1개의 답
x_train=tmp_train.drop(['Auction_key','Hammer_price','point.x','point.y','Current_floor'],axis=1)
y_train=tmp_train['Hammer_price']
x_test=tmp_test.drop(['Auction_key','Hammer_price','point.x','point.y','Current_floor'],axis=1)

In [8]:
from sklearn.preprocessing import StandardScaler

encoder=StandardScaler()

for col in x_train.columns:
    x_train[col]=encoder.fit_transform(x_train[col].values.reshape(-1,1))
    x_test[col]=encoder.fit_transform(x_test[col].values.reshape(-1,1))
display(x_train.head())
display(x_test.head())

,Claim_price,Auction_count,Auction_miscarriage_count,Total_land_gross_area,Total_land_real_area,Total_land_auction_area,Total_building_area,Total_building_auction_area,Total_appraisal_price,Minimum_sales_price,Total_floor
0,1.043456,0.174819,0.254466,-0.233008,-0.076371,-0.060591,0.802970,0.820282,0.430194,0.503488,2.947348
1,-0.162572,0.174819,0.254466,0.083594,-0.361243,-0.344977,0.206615,0.226845,0.731268,0.880500,5.577111
2,-0.268455,1.240831,1.457112,1.219281,0.439278,0.454179,-0.437249,-0.413869,-0.480651,-0.674955,-0.208368
3,-0.139668,0.174819,0.254466,-0.107866,-0.143337,-0.127442,-0.108261,-0.086491,-0.265452,-0.367618,0.317585
4,-0.242615,0.174819,0.254466,-0.358203,0.043615,0.059191,-0.107791,-0.086023,-0.415862,-0.555965,-1.049892


,Claim_price,Auction_count,Auction_miscarriage_count,Total_land_gross_area,Total_land_real_area,Total_land_auction_area,Total_building_area,Total_building_auction_area,Total_appraisal_price,Minimum_sales_price,Total_floor
0,0.490222,5.34605,5.950794,-0.647578,-0.086055,-0.959754,0.370874,0.405242,-0.539687,-0.837689,-0.267088
1,0.490222,4.30497,4.800325,-0.647578,-0.089839,-1.033034,0.137139,0.176709,-0.577740,-0.790260,-0.267088
2,0.490222,5.34605,5.950794,-0.647578,-0.086055,-0.959754,0.370874,0.405242,-0.527660,-0.831956,-0.267088
3,0.490222,5.34605,5.950794,-0.647578,-0.086055,-0.959754,0.370874,0.405242,-0.527660,-0.831956,-0.267088
4,0.490222,5.34605,5.950794,-0.647578,-0.086055,-0.959754,0.370874,0.405242,-0.527660,-0.831956,-0.267088


In [9]:
# 데이터 정의
x_train=torch.FloatTensor(x_train.values)
y_train=torch.FloatTensor(y_train.values)

dataset=TensorDataset(x_train,y_train)
dataloader=DataLoader(dataset,batch_size=10,shuffle=True)

# 모델 초기화
model=nn.Linear(11,1)

# optimizer 설정 
optimizer = optim.SGD(model.parameters(),lr=0.001)

# 원하는만큼 경사 하강법을 반복
for epoch in range(101):
    for batch_idx,samples in enumerate(dataloader):
        # 데이터
        x_train,y_train=samples
        # 가설
        y_pred=model(x_train)
        # cost 계산
        cost=F.mse_loss(y_pred,y_train)
        # cost로 가설 개선
        optimizer.zero_grad() # 미분 누적 초기화
        cost.backward() # 미분값 자동 계산
        optimizer.step() # w,b gradient 업데이트.
        # 로그 출력
        print(f"Epoch : {epoch}/100, batch : {batch_idx+1}/{len(dataloader)}, Cost : {cost.item()}")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Epoch : 0/100, batch : 1/194, Cost : 8.869030500426056e+17
Epoch : 0/100, batch : 2/194, Cost : 5.282496413266084e+17
Epoch : 0/100, batch : 3/194, Cost : 1.179504362871849e+17
Epoch : 0/100, batch : 4/194, Cost : 4.247863187073925e+17
Epoch : 0/100, batch : 5/194, Cost : 1.6630582380540723e+17
Epoch : 0/100, batch : 6/194, Cost : 2.162997241671844e+17
Epoch : 0/100, batch : 7/194, Cost : 1.572107151600517e+17
Epoch : 0/100, batch : 8/194, Cost : 1.8255464715884954e+17
Epoch : 0/100, batch : 9/194, Cost : 2.92810048239829e+17
Epoch : 0/100, batch : 10/194, Cost : 1.3583291058120294e+18
Epoch : 0/100, batch : 11/194, Cost : 4.007013447022674e+17
Epoch : 0/100, batch : 12/194, Cost : 4.378954522479821e+17
Epoch : 0/100, batch : 13/194, Cost : 6.122204009018163e+16
Epoch : 0/100, batch : 14/194, Cost : 2.321048375416534e+19
Epoch : 0/100, batch : 15/194, Cost : 2.643221486407516e+17
Epoch : 0/100, batch : 16/194, Cost : 4.179851177350922e+17
Epoch : 0/100, batch : 17/194, Cost : 1.6572108

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch : 75/100, batch : 45/194, Cost : 7.506917356627558e+16
Epoch : 75/100, batch : 46/194, Cost : 1.3875608250272973e+17
Epoch : 75/100, batch : 47/194, Cost : 7.685357785894093e+16
Epoch : 75/100, batch : 48/194, Cost : 1.0657526694333645e+17
Epoch : 75/100, batch : 49/194, Cost : 6.027246577070899e+16
Epoch : 75/100, batch : 50/194, Cost : 4.441633986707456e+16
Epoch : 75/100, batch : 51/194, Cost : 5.4865561506545664e+17
Epoch : 75/100, batch : 52/194, Cost : 5.040894882139341e+16
Epoch : 75/100, batch : 53/194, Cost : 4.749043264441549e+16
Epoch : 75/100, batch : 54/194, Cost : 6.818946059180442e+16
Epoch : 75/100, batch : 55/194, Cost : 5.061580733127066e+16
Epoch : 75/100, batch : 56/194, Cost : 5.048284373372109e+16
Epoch : 75/100, batch : 57/194, Cost : 4.897096971887575e+17
Epoch : 75/100, batch : 58/194, Cost : 1.2130303621883494e+17
Epoch : 75/100, batch : 59/194, Cost : 7.91538764433326e+16
Epoch : 75/100, batch : 60/194, Cost : 4.38454

In [13]:
pred=model(torch.FloatTensor(x_test.values))
submission['Hammer_price']=pred.detach().numpy()
submission.head()

,Auction_key,Hammer_price
0,1778,439333728.0
1,1779,438463264.0
2,1784,439859040.0
3,1786,439859040.0
4,1790,439859040.0


In [14]:
submission.to_csv('pytorch_20200113.csv',index=False)